# Metal Segregation
### With rhyolite-MELTS and $\mu$O<sub>2</sub> constrained using the empirical method of Kress and Carmichael (1991)

This notebook demonstates the use of the Equilibrate class in the equilibrate module to perform equilibrium crystallization clculations for the special case of oxygen fugacity constrained using the model of Kress and Carmichael (1991).  This model is applicable to natural composition silicate liquids and is usually applied in conjunction with thermodynamic models published by Ghiorso and Sack (1995) [MELTS], Ghiorso et al., (2003) [pMELTS], Gualda et al. (2012) [rhyolite-MELTS] or Ghiorso and Gualda (2015) [rhyoliteMELTS + CO2].



In [ ]:
import numpy as np
import scipy.optimize as opt
import scipy.linalg as lin 
import sys
import matplotlib.pyplot as plt

In [ ]:
from thermoengine import core, phases, model, equilibrate

## Create phases for equilibrium assemblages
Minimal rhyolitic phase assemblage

In [ ]:
modelDB = model.Database(liq_mod='v1.0')

In [ ]:
Liquid = modelDB.get_phase('Liq')
MetalL = modelDB.get_phase('MtlL')
MetalS = modelDB.get_phase('MtlS')

## Define elements in system and phases in system
These 15 elements are required for the silicate liquid model of MELTS (rhyolite-MELTS)

In [ ]:
elm_sys = ['H','O','Na','Mg','Al','Si','P','K','Ca','Ti','Cr','Mn','Fe','Co','Ni']
phs_sys = [Liquid, MetalL, MetalS]

## Composition of the system
This composition chosen as close to mantle comp???
Used in exoMELTS proposal

In [ ]:
grm_oxides = {
    'SiO2':44.62,   
    'TiO2':0.16,
    'Al2O3':3.49,
    'Fe2O3':0.51,
    'Cr2O3':0.38,
    'FeO':5.64,
    'MnO':0.00,
    'MgO':39.21,
    'NiO': 2.88,
    'CoO': 0.00,
    'CaO': 2.77,
    'Na2O': 0.30,
    'K2O': 0.02,
    'P2O5': 0.01,
    'H2O': 0,
    'CO2': 0,
}

In [ ]:
tot_grm_oxides = 0.0
for key in grm_oxides.keys():
    tot_grm_oxides += grm_oxides[key]

Cast this composition as moles of elements for input to the Equilibrate class

In [ ]:
mol_oxides = core.chem.format_mol_oxide_comp(grm_oxides, convert_grams_to_moles=True)
moles_end,oxide_res = Liquid.calc_endmember_comp(
    mol_oxide_comp=mol_oxides, method='intrinsic', output_residual=True)
if not Liquid.test_endmember_comp(moles_end):
    print ("Calculated composition is infeasible!")
mol_elm = Liquid.covert_endmember_comp(moles_end,output='moles_elements')

In [ ]:
blk_cmp = []
for elm in elm_sys:
    index = core.chem.PERIODIC_ORDER.tolist().index(elm)
    blk_cmp.append(mol_elm[index])
blk_cmp = np.array(blk_cmp)

In [ ]:
blk_cmp

# Run the calculation

In [ ]:
equil = equilibrate.Equilibrate(elm_sys, phs_sys)

Uncomment here and in the next cell to generate a call graph that traces execution times by method and module 

In [ ]:
#%load_ext snakeviz

### For reference, equilibrate at an initial temperature and pressure

In [ ]:
#%%snakeviz --new-tab
T = 1000+273.15
P = 1.0
state = equil.execute(T, P, bulk_comp=blk_cmp, debug=0, stats=False)
state.print_state()

Compute the system oxygen fugacity using Kress and Carmichael (1991) along with the total moles of oxygen in the system

In [ ]:
results = []
results.append((T-273.15,P/10.0) + equil.kc_print_state(state))

### Assign an oxygen buffer, and recalculate ...

In [ ]:
buffer = 'IW'
dlogfO2 = -5

dNNO = (dlogfO2+modelDB.redox_buffer(T, P, buffer=buffer)
        -modelDB.redox_buffer(T, P, buffer='NNO'))
dNNO

In [ ]:
state = equil.execute(T, P, bulk_comp=blk_cmp, con_deltaNNO=dNNO, 
                      debug=0, stats=False)
state.print_state()

In [ ]:
results.append((T-273.15,P/10.0) + equil.kc_print_state(state))

In [ ]:
mass_silicate = state.tot_grams_phase('Liquid')
mass_alloy = (state.tot_grams_phase('Solid Alloy')
              +state.tot_grams_phase('Liquid Alloy'))

In [ ]:
alloy_frac = mass_alloy/(mass_silicate+mass_alloy)
alloy_frac

In [ ]:
dlogfO2 = np.arange(-8,8.1,2)
dNNO_buf = (modelDB.redox_buffer(T, P, buffer=buffer)
            -modelDB.redox_buffer(T, P, buffer='NNO'))

alloy_frac = []
for idlogfO2 in dlogfO2:
    dNNO = dNNO_buf + idlogfO2
    state = equil.execute(T, P, bulk_comp=blk_cmp, 
                          con_deltaNNO=dNNO, 
                          debug=0, stats=False)
    mass_silicate = state.tot_grams_phase('Liquid')
    mass_alloy = (state.tot_grams_phase('Solid Alloy')
                  +state.tot_grams_phase('Liquid Alloy'))
    ialloy_frac = mass_alloy/(mass_silicate+mass_alloy)
    
    alloy_frac.append(ialloy_frac)
    
alloy_frac = np.array(alloy_frac)
    


In [ ]:
plt.figure()
plt.plot(dlogfO2, alloy_frac, 'k-')
plt.xlabel('$\Delta$ IW')
plt.ylabel('Fe-Alloy Mass Fraction')